# BF528 Project 4 Final Report
## Fred Choi

### Introduction:
Dendritic cells rely on coordinated transcriptional and epigenetic programs to establish their distinct subsets and to support immunity. EBut how chromatin modifications actually changes these processes is not as well understood. The study was performed to define the role of the histone deacetylation in dendritic cell differentiation and to determine how its loss alters gene expression, chromatin accessibility, and downstream immune function. The authors used RNA-seq to capture transcriptional changes caused by HDAC1 deletion, ATAC-seq to measure accessibility changes at regulatory regions that control DC subset specification, and Cut&Run to assess shifts in histone acetylation at those same loci. These bioinformatic approaches allow the authors to connect HDAC1-dependent chromatin remodeling with specific transcriptional outputs and potentially guide DC-targeted therapeutic strategies for immuno-oncology.

### Methods:
Raw single-end reads were downloaded from the paper using the GEO accession number and included both wildtype (WT) and knockout (KO) replicates for 2 different cell types (DC1 and DC2). Read quality was first assessed with FastQC (version 0.12.0) to act as a quality control check on the raw sequence data. Low-quality bases and adapter sequences were removed using Trimmomatic (version 0.40) for each replicate. A genome index was generated using Bowtie2-build (version 2.5.4) from the mm10 reference genome. Trimmed reads were then aligned to the indexed genome using Bowtie2 with the -very-sensitive paramter to produce BAM alignment files. The BAM files from Bowtie2-align were sorted by genomic coordinate using Samtools sort and then indexed with Samtools to enable efficient access for downstream analyses. Mitochondrial DNA was removed from each sorted BAM file using Samtools view by filtering out reads that mapped to “chrM.” Alignment statistics were also calculated for each sample using Samtools flagstat (version 1.19.1) to report summary metrics such as total reads, mapped reads, and overall mapping efficiency. All quality control results were summarized with MultiQC (version 1.32) using the outputs from FastQC, Trimmomatic, and Samtools flagstat.

Peaks were identified with MACS3 callpeak (version 3.0.4) using the “--nomodel,” “--keep-dup auto,” and “--extsize 147” settings stated in the original paper. MACS3 was run separately for each WT and KO replicate in both DC1 and DC2 cells. The resulting peak files were then imported into R for differential accessibility analysis between WT and KO samples for each cell type with DiffBind. Read counts were normalized using TMM normalization from edgeR. Peaks were filtered using a p-value threshold of 0.01 and then exported as BED files for downstream analyses. The filtered peaks were annotated with HOMER annotatePeaks.pl using the mm10 reference genome and corresponding GTF file to assign each peak to nearby genes or regulatory features. Motif enrichment was also assessed with HOMER findMotifsGenome.pl using the mm10 reference genome to identify enriched DNA sequence motifs within the filtered peak set.

Normalized bigWig files were generated from the mitochondrial-filtered BAMs with deepTools bamCoverage (version 3.5.6) for signal visualization. To assess accessibility around transcription start sites, deepTools computeMatrix was run in reference-point mode using mm10 gene annotations with 2000 bp added to the start and end sites as padding. A second computeMatrix run was performed using the differential peak BED files along with the bigWig tracks to produce matrices for heatmap visualization. Heatmaps of gain and loss regions were generated with deepTools plotHeatmap to compare signal patterns across replicates.

### Quality Control Evaluation:
For quality control, the MultiQC results show that overall sequencing quality is strong across all samples with Phred scores above 30 and no issues with GC content or duplication levels. Some potential issues did come up from the FASTQC results. Adapter signals were detected mainly at the ends of reads, but this is expected for an ATAC-seq experiment due to short fragment sizes. Trimmomatic successfully removed these adapters, and trimming did not cause major read loss. The per base sequence content “failed” on all samples but this was only at the beginning of the sequences which is very common to see and explained by technical bias. There were overrepresented sequences found in the DC2 samples, but these could be adapters or mitochondrial DNA that were later removed downstream and only took up less than 1% of total reads. All things considered, since all other major QC metrics appeared good, I believe these results indicate that the sequencing data is of high quality and suitable for downstream analysis without any additional steps.

### Alignment Statistics Evaluation:
The alignment statistics (flagstats) show strong mapping rates across samples with roughly 25-28 million reads uniquely mapped in almost all samples which originally had roughly 20-30 million reads. One of the samples (DC2 WT rep 2) has slighly reads than the others with about 19.6 million reads. This could simply be explained by technical factors such as the sample receiving fewer raw reads during sequencing or higher loss of reads during trimming. But samples with slightly lower mapped reada still retained enough usable reads for downstream analysis as they also passed quality control. The high alignment percentages would suggest successful capture of accessible chromatin regions and the data can be used for downstream analysis.


### Deliverables:

#### ATAC-seq QC metrics and DARs
Through EdgeR analysis, 2,785 differentially accessible regions were found in DC1 and 1,500 in DC2. This would suggest that HDAC KO had an effect on chromatin accessibility on both cell types (more so with DC1 cells). The two ATAC-seq QC metrics I looked at were TSS enrichment score and FRiP. The TSS enrichment scores ranged from about 4.3 to 5.0, support that there is good quality signal at transcription start sites. High TSS enrichment indicates clear enrichment of accessible chromatin regions at active promoters. The FRiP values were also consistent across samples with scores between 0.18 and 0.22. The higher FRiP values indicate low background noise and confirm that the dataset contains meaningful biological signal. Together, these two QC metrics show that the experiment produced reliable accessibility profiles that are well suited for downstream analyses.

![TSS_Score](pictures/TSS_Score.png)
![DC1_FRIP](pictures/DC1_FRIP.png)
![DC2_FRIP](pictures/DC2_FRIP.png)

#### Enrichment Analysis
The enrichment analysis showed that the differentially accessible regions in both DC1 and DC2 mapped to pathways involved in development, cellular regulation, and cell differentiation. These results suggest that HDAC1 loss may influence accessibility in pathways that control how dendritic cells develop and acquire their specific identities. There weren't many differences between enriched pathways between DC1 and DC2 cells either.

![DC1_Enrichment](pictures/DC1_Enrichment.png)
![DC2_Enrichment](pictures/DC2_Enrichment.png)

#### Motif Analysis
Motif analysis supported these findings by identifying strong enrichment for transcription factors such as PU.1, ELF, and Spib factors with p-values as low as 1e-300 and were present in roughly 20-30% of target sequences. The PU.1 motif had the highest enrichment in both DC1 and DC2 and occurred in about 28–30% of target sequences. These motifs were found to be regulators of dendritic cell development and differentiation. Additional enriched motifs included BATF, Fra1, and FOSL1 which are linked to immune activation and AP-1–mediated enhancer activity. The presence of motifs related to development/differentiation and immune signaling suggests that HDAC1 loss affects accessibility in regions controlled by core developmental transcription factors as well as broader regulatory pathways that shape immune cell activity.

![DC1_Motifs](pictures/DC1_Motifs.png)
![DC2_Motifs](pictures/DC2_Motifs.png)


#### Figure 6A-6F Recreation
I believe overall reproduction of figures 6A-F were successful and support the same biological interpretations with only minor differences. The heatmaps from 6A and 6B show accessibility gains and losses in DC1 and DC2 cells with very similar patterns as the paper. The central regions display accessibility shifts between WT and KO samples as well. Figures 6C and 6E also follow the general trend of the paper. There were some differences on where the paper's genes were found on the graph, but this could be explained by different thresholds used to define differential expression, variation in normalization, or even difference in peak calling function (MACS2 vs MACS3). But the general trend is the same where we see higher expression and/or chromatin accessibility of the target genes that the paper looked into. Figures 6D and 6F I believe were very close to those from the paper where the same peak patterns are found at the same expected locations when comparing to the reference sequence. Overall for all figures, any minor differences found could be explained by difference in version number of tools being used or general workflow parameters since they were not completely given in the paper.

![Figure6A](pictures/Figure6A.png)
![Figure6B](pictures/Figure6B.png)
![Figure6C](pictures/Figure6C.png)
![Figure6E](pictures/Figure6E.png)
![Figure6D](pictures/Figure6D.png)
![Figure6F](pictures/Figure6F.png)